# Purpose

### Author: Julius Remigio

Split reviews.csv by category, which can be found in the metadata file.

use pattern from here:
https://github.com/DSE-capstone-sharknado/models-legacy
reviews.csv into men, women, phone etc like that cpp file does

In [1]:
import pandas as pd
import gzip
import shutil

In [2]:
# categories we are interested in 
categories = ["Women", "Men", "Girls", "Boys", "Baby"]

In [3]:
# Reading the metadat file
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

### Review Columns
    reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
    asin - ID of the product, e.g. 0000013714
    unixReviewTime - time of the review (unix time)
   


In [4]:
# create dataframe from reviews.csv
reviews = pd.read_csv('reviews.csv', names=['reviewerID', 'asin', 'unixReviewTime'])
reviews.head()

,reviewerID,asin,unixReviewTime
0,A1KLRMWW2FWPL4,0000031887,1297468800
1,A2G5TCU2WDFZ65,0000031887,1358553600
2,A1RLQXYNCMWRWN,0000031887,1357257600
3,A8U3FAMSJVHS5,0000031887,1398556800
4,A3GEOILWLK86XM,0000031887,1394841600


In [5]:
# read the data file, it takes a few minutes
df_meta = getDF('meta_Clothing_Shoes_and_Jewelry.json.gz')

In [6]:
print(df_meta.columns)
print(df_meta.shape)

Index([u'asin', u'categories', u'title', u'price', u'salesRank', u'imUrl',
       u'brand', u'related', u'description'],
      dtype='object')
(1503384, 9)


In [34]:
print(df_meta.brand.describe())
print 'Percent NaN:', 1- df_meta.brand.describe()['count']/float(df_meta.shape[0])


count                      98132
unique                      8501
top       Rubie&#39;s Costume Co
freq                        4079
Name: brand, dtype: object
Percent NaN: 0.93472592498


In [8]:
# we only care about brand and categories columns
df_meta = df_meta[[u'asin', u'categories',u'brand']]

# categories is a list of lists 
df_meta[:1].categories[0]

[['Clothing, Shoes & Jewelry', 'Girls'],
 ['Clothing, Shoes & Jewelry',
  'Novelty, Costumes & More',
  'Costumes & Accessories',
  'More Accessories',
  'Kids & Baby']]

In [9]:
df_meta.head()

,asin,categories,brand
0,0000037214,"[[Clothing, Shoes & Jewelry, Girls], [Clothing...",Big Dreams
1,0000031887,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie
2,0123456479,"[[Clothing, Shoes & Jewelry, Novelty, Costumes...",NaN
3,0456844570,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",NaN
4,0456808574,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",NaN


In [10]:
# get distinct categories - we only care about the second level of the first list
sorted([c for c in df_meta.categories.apply(lambda x: x[0][1]).drop_duplicates().tolist() if c[0] in 'WMGB'])

['B',
 'Baby',
 'Baby & Child Care',
 'Baby & Toddler Toys',
 'Bath',
 'Bath & Body',
 'Bathing & Skin Care',
 'Beading & Jewelry-Making',
 'Bedding',
 'Beverages',
 'Blues',
 'Boating & Water Sports',
 'Boot Shop',
 'Boutique Designer Jewelry',
 "Boy's Athletic Watches",
 'Boys',
 'Breitling Watches',
 'Broadway & Vocalists',
 'Building Toys',
 'G',
 'GPS & Navigation',
 'Game Time',
 'Games',
 'Gardening & Lawn Care',
 'Gear',
 'Gemstones',
 'Geography',
 'Gifts',
 'Girls',
 'Glycine',
 'Golf',
 'Grown-Up Toys',
 'M',
 'Made in USA',
 'Makeup',
 'Material Handling Products',
 'Medical Supplies & Equipment',
 'Men',
 "Men's Athletic Watches",
 'Michael Kors Watches',
 'Michele Watches',
 'Motorcycle & ATV Casual Footwear',
 'Motorcycle & Powersports',
 'Movado',
 'Movies',
 'W',
 'Wedding Party Gifts',
 'Winter Promo',
 'Wolf Designs',
 'Women',
 "Women's Athletic Watches",
 "Women's Luxury Brands"]

In [11]:
# join metadata and reviews so we can split the reviews by category
combined = pd.merge(reviews, df_meta, left_on='asin', right_on='asin')
combined.head()

,reviewerID,asin,unixReviewTime,categories,brand
0,A1KLRMWW2FWPL4,0000031887,1297468800,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie
1,A2G5TCU2WDFZ65,0000031887,1358553600,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie
2,A1RLQXYNCMWRWN,0000031887,1357257600,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie
3,A8U3FAMSJVHS5,0000031887,1398556800,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie
4,A3GEOILWLK86XM,0000031887,1394841600,"[[Clothing, Shoes & Jewelry, Girls, Clothing, ...",Boutique Cutie


In [17]:
# create a file for each category of reviews.
for c in categories:
    csv = 'review_{0}.csv'.format(c)
    csvc = 'review_{0}c.csv'.format(c)
    
    # create files without the categories column
    combined[combined.categories.apply(lambda x: x[0][1])==c][[c for c in combined.columns if c != 'brand']].to_csv(csv + '.gz', compression='gzip',index=False)
    
    # create files with the categories column
    combined[combined.categories.apply(lambda x: x[0][1])==c].to_csv(csvc + '.gz', compression='gzip',index=False)